In [1]:
import torch
import utulek
import numpy as np

In [2]:
hyper = {
    "batch_size": 128,
    "epochs": 64,
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

In [3]:
class Cifar10Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        self.X, self.Y = utulek.fetch_tfds_img_ds("cifar10", split)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [4]:
result = {
    "c10": {
        "train": Cifar10Dataset("train"),
        "test": Cifar10Dataset("test")
    }
}

In [5]:
result["c10"].update({
    "train-loader": torch.utils.data.DataLoader(result["c10"]["train"], batch_size=hyper["batch_size"], pin_memory=True),
    "test-loader": torch.utils.data.DataLoader(result["c10"]["test"], batch_size=hyper["batch_size"], pin_memory=True)
})

for X, Y in result["c10"]["train-loader"]:
    print(f"X \in {X.shape}")
    print(f"Y \in {Y.shape} of {Y.dtype}")
    break

X \in torch.Size([128, 32, 32, 3])
Y \in torch.Size([128]) of torch.int64


In [6]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(32**2*3, 4096),
            torch.nn.ReLU(),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(),
            torch.nn.Linear(4096, 10)
        )

    def forward(self, X):
        X = self.flatten(X)
        logits = self.linear_relu_stack(X)
        return logits


print(model := NeuralNetwork().to(hyper["device"]))

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=4096, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4096, out_features=4096, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4096, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=3e-4)


def train(loader, model, loss_fn, optimizer):
    model.train()
    
    data = iter(loader)
    XY_next = data.next()
    XY_next = [i.to(hyper["device"], non_blocking=True) for i in XY_next]
    
    for i in range(len(loader)):
        X, Y = XY_next
        if i + 1 != len(loader):
            XY_next = data.next()
            XY_next = [i.to(hyper["device"], non_blocking=True) for i in XY_next]
        
        def update():
            pred = model(X)
            loss = loss_fn(pred, Y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        utulek.time("update", update)


def test(loader, model, loss_fn, epoch):
    model.eval()
    
    size = len(loader.dataset)
    num_batches = len(loader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, Y in loader:
            X, Y = X.to(hyper["device"]), Y.to(hyper["device"])
            pred = model(X)
            test_loss += loss_fn(pred, Y).item()
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"{epoch}: {(100*correct):>0.1f}% | {test_loss:>8f}")

In [8]:
%%time
for epoch in range(hyper["epochs"]):
    train(result["c10"]["train-loader"], model, loss_fn, optimizer)
    utulek.time("test", test, result["c10"]["test-loader"], model, loss_fn, epoch)
    
print(utulek.time_result)

0: 10.9% | 2.291881
1: 11.7% | 2.280636
2: 13.8% | 2.270380
3: 16.1% | 2.260395
4: 17.9% | 2.250452
5: 19.5% | 2.240403
6: 20.8% | 2.230154
7: 22.0% | 2.219658
8: 23.0% | 2.208914
9: 23.7% | 2.197956
10: 24.6% | 2.186819
11: 25.3% | 2.175541
12: 25.9% | 2.164193
13: 26.2% | 2.152847
14: 26.8% | 2.141577
15: 27.3% | 2.130464
16: 27.7% | 2.119576
17: 28.0% | 2.108983
18: 28.2% | 2.098731
19: 28.5% | 2.088852
20: 28.8% | 2.079360
21: 29.2% | 2.070257
22: 29.4% | 2.061535
23: 29.7% | 2.053176
24: 29.8% | 2.045163
25: 29.9% | 2.037474
26: 30.2% | 2.030086
27: 30.3% | 2.022990
28: 30.5% | 2.016162
29: 30.8% | 2.009585
30: 30.8% | 2.003243
31: 30.9% | 1.997124
32: 31.2% | 1.991221
33: 31.3% | 1.985524
34: 31.4% | 1.980022
35: 31.4% | 1.974712
36: 31.6% | 1.969582
37: 31.6% | 1.964624
38: 31.8% | 1.959833
39: 31.9% | 1.955199
40: 32.0% | 1.950716
41: 32.2% | 1.946377
42: 32.3% | 1.942175
43: 32.4% | 1.938104
44: 32.5% | 1.934158
45: 32.7% | 1.930332
46: 32.9% | 1.926621
47: 33.0% | 1.923017
48

In [9]:
for X, Y in dss["c10-dl"][0]:
    pred = model(X.to(hps["device"])).argmax(axis=1).cpu()
    utulek.show_img_ds(X, Y, pred, labels=utulek.fetch_tfds_img_ds_labels("cifar10"), shape=(6, 5))
    break

NameError: name 'dss' is not defined